<a href="https://colab.research.google.com/github/StevenMena/03MIAR---Algoritmos-de-Optimizacion/blob/main/Busqueda_Tabu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import random

# Define los parámetros del problema
NUM_ACTORES = 5
NUM_TOMAS = 15
NUM_MAX_TOMAS_DIA = 6
NUM_ITERACIONES = 100
NUM_VECINOS = 10
NUM_TABU = 5

# Define la matriz que relaciona los actores con las tomas
MATRIZ_ACTORES_TOMAS = [[1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
                        [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
                        [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
                        [0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
                        [0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1]]

# Función para calcular el costo de una solución
def calcular_costo(solucion):
    costo = 0
    for i in range(len(solucion)):
        actores_presentes = [j for j in range(NUM_ACTORES) if MATRIZ_ACTORES_TOMAS[j][i] == 1]
        num_actores_presentes = len(actores_presentes)
        if num_actores_presentes > 0:
            costo += 1/num_actores_presentes
    return costo

# Genera una solución inicial aleatoria
def generar_solucion_inicial():
    solucion = []
    for i in range(NUM_TOMAS):
        solucion.append(random.sample(range(NUM_ACTORES), k=random.randint(1, min(NUM_MAX_TOMAS_DIA, NUM_ACTORES))))
    return solucion

# Genera vecinos de una solución
def generar_vecinos(solucion):
    vecinos = []
    for i in range(NUM_VECINOS):
        vecino = solucion.copy()
        indice_toma = random.randint(0, NUM_TOMAS-1)
        indice_actor = random.randint(0, NUM_ACTORES-1)
        if indice_actor in vecino[indice_toma]:
            vecino[indice_toma].remove(indice_actor)
        else:
            if len(vecino[indice_toma]) < NUM_MAX_TOMAS_DIA:
                vecino[indice_toma].append(indice_actor)
        vecinos.append(vecino)
    return vecinos



# Función de búsqueda tabú
def busqueda_tabu():
    mejor_solucion = generar_solucion_inicial()
    mejor_costo = calcular_costo(mejor_solucion)
    solucion_actual = mejor_solucion.copy()
    costo_actual = mejor_costo
    lista_tabu = []
    for i in range(NUM_ITERACIONES):
        vecinos = generar_vecinos(solucion_actual)
        mejor_vecino = None
        mejor_costo_vecino = float('inf')
        for vecino in vecinos:
            costo_vecino = calcular_costo(vecino)
            if costo_vecino < mejor_costo_vecino and vecino not in lista_tabu:
                mejor_vecino = vecino
                mejor_costo_vecino = costo_vecino
        if mejor_vecino is None:
            break
        solucion_actual = mejor_vecino
        costo_actual = mejor_costo_vecino
        if costo_actual < mejor_costo:
            mejor_solucion = solucion_actual
            mejor_costo = costo_actual
        lista_tabu.append(mejor_vecino)
        if len(lista_tabu) > NUM_TABU:
            lista_tabu.pop(0)
    return mejor_solucion, mejor_costo

# Ejecuta la búsqueda tabú y muestra los resultados
mejor_solucion, mejor_costo = busqueda_tabu()
print("Mejor solución encontrada: ", mejor_solucion)
print("Costo de la mejor solución: ", mejor_costo)

Mejor solución encontrada:  [[0, 2], [4, 1, 0], [4, 0], [3, 4, 1], [2, 3, 0], [1, 4, 0], [4, 1, 0], [1, 3, 0, 2], [3, 4], [0, 2, 4], [1, 4, 2, 0], [], [2, 1, 0], [2, 4], [0, 4]]
Costo de la mejor solución:  4.866666666666666


En esta implementación, la función busqueda_tabu() ejecuta la búsqueda tabú para encontrar la mejor solución para el problema de planificación de tomas de una película. La función utiliza una solución inicial aleatoria y genera vecinos de esta solución. Luego, se evalúa el costo de cada vecino y se elige el mejor vecino como la siguiente solución. La función también utiliza una lista tabú para evitar caer en ciclos de soluciones repetidas.

Finalmente, la función devuelve la mejor solución encontrada y su costo, que se muestran en la pantalla al ejecutar el código.

Otra solucion usando numpy

In [12]:
import numpy as np

# Configuración del problema
num_actors = 5
num_takes = 15
max_takes_per_day = 6
num_days = int(np.ceil(num_takes / max_takes_per_day))
cost_per_actor_per_day = 1000

num_iterations = 100
tabu_list = []
best_solution = None
best_cost = np.inf

# Matriz de relaciones entre actores y tomas
# Cada fila representa un actor y cada columna representa una toma
# El valor en la posición (i,j) indica si el actor i aparece en la toma j

takes_matrix = np.array([[1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
                        [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
                        [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
                        [0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
                        [0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1]])

# Función para calcular el costo de una solución
def calculate_cost(solution):
    """
    La solución es una matriz booleana de tamaño (num_actors, num_takes)
    que indica para cada actor y cada toma si el actor aparece en la toma.
    """
    # Separar las tomas en grupos de máximo max_takes_per_day tomas por día
    takes_per_day = [solution[:, i:i+max_takes_per_day] for i in range(0, num_takes, max_takes_per_day)]

    # Calcular el costo total sumando el costo por actor por día
    total_cost = 0
    for takes in takes_per_day:
        actors_in_takes = np.sum(takes, axis=1) > 0
        total_cost += np.sum(actors_in_takes) * cost_per_actor_per_day

    return total_cost

# Función para generar una solución inicial aleatoria
def generate_random_solution():
    """
    Genera una matriz booleana de tamaño (num_actors, num_takes)
    que indica para cada actor y cada toma si el actor aparece en la toma.
    """
    solution = np.zeros((num_actors, num_takes), dtype=bool)
    for i in range(num_takes):
        available_actors = np.where(takes_matrix[:, i])[0]
        np.random.shuffle(available_actors)
        for actor in available_actors:
            if np.sum(solution[actor]) < max_takes_per_day:
                solution[actor, i] = True
                break
    return solution


# Función para generar una solución vecina
def generate_neighbor_solution(current_solution):
    """
    Genera una solución vecina a partir de la solución actual.
    """
    # Copiar la solución actual
    neighbor_solution = current_solution.copy()

    # Elegir aleatoriamente un actor y una toma que estén presentes en la solución
    actor_indices, take_indices = np.nonzero(current_solution)
    random_index = np.random.choice(len(actor_indices))
    actor_index, take_index = actor_indices[random_index], take_indices[random_index]

    # Quitar al actor de la toma actual
    neighbor_solution[actor_index, take_index] = False

    # Elegir aleatoriamente una toma disponible para el actor y agregarlo
    available_takes = np.where(takes_matrix[actor_index])[0]
    np.random.shuffle(available_takes)
    for available_take in available_takes:
        if np.sum(neighbor_solution[actor_index]) < max_takes_per_day:
            neighbor_solution[actor_index, available_take] = True
            break

    return neighbor_solution


# Algoritmo de búsqueda tabú

def busqueda_tabu():
    current_solution = generate_random_solution()
    current_cost = calculate_cost(current_solution)
    best_cost = current_cost
    for i in range(num_iterations):
        # Generar una solución vecina que no esté en la lista tabú
        neighbor_solution = generate_neighbor_solution(current_solution)
        while np.array2string(neighbor_solution) in tabu_list:
            neighbor_solution = generate_neighbor_solution(current_solution)

        # Calcular el costo de la solución vecina
        neighbor_cost = calculate_cost(neighbor_solution)

        # Actualizar la mejor solución encontrada hasta el momento
        if neighbor_cost < best_cost:
            best_solution = neighbor_solution
            best_cost = neighbor_cost

        # Actualizar la solución actual y la lista tabú
        current_solution = neighbor_solution
        tabu_list.append(np.array2string(current_solution))
        if len(tabu_list) > 10:
            tabu_list.pop(0)

    print("Mejor solución encontrada:")
    print(best_solution)
    print("Costo de la mejor solución encontrada:")
    print(best_cost)


busqueda_tabu()

Mejor solución encontrada:
[[False False False False False False False False False False False False
  False False False]
 [False False False  True False False False False False False False False
  False False False]
 [False False False False False False False  True False False False False
  False False False]
 [False False False False False False  True False  True False False False
  False False False]
 [False  True  True False False False False False False False False False
  False False False]]
Costo de la mejor solución encontrada:
4000


La función generate_neighbor_solution elige aleatoriamente una toma y un actor que estén presentes en la solución actual, quita al actor de esa toma y elige aleatoriamente una toma disponible para ese actor y lo agrega a la solución. De esta manera se genera una solución vecina.

Finalmente, para encontrar la solución óptima, podemos utilizar el algoritmo de búsqueda tabú:


El algoritmo de búsqueda tabú comienza generando una solución aleatoria y calculando su costo. Luego, en cada iteración, genera una solución vecina que no esté en la lista tabú, calcula su costo y actualiza la mejor solución encontrada hasta el momento. Finalmente, actualiza la solución actual y la lista tabú. El algoritmo se ejecuta durante un número fijo de iteraciones (en este caso, 100).

La mejor solución encontrada y su costo se imprimen al final del algoritmo. Esta solución muestra para cada actor y para cada toma si el actor aparece o no en la toma. La solución encontrada minimiza el costo total de los actores por día, cumpliendo con la